In [1]:
import os
import sys
import ipympl

In [2]:
print(os.getcwd())
sys.path.insert(0, os.path.join(os.getcwd(), '../'))

/home/haruyaishikawa/dev/RecordWithRS/python/notebook


In [3]:
import numpy as np
from datetime import datetime as dt
from utils import DataManagement

In [21]:
# setup which datetime to get
dm = DataManagement()
after = dt(2018, 9, 9, 13, 7, 0)
before = dt(2018, 9, 9, 13, 8, 0)
datetimes = dm.get_datetimes_in(after, before)

assert len(datetimes) == 1
datetime = datetimes[0]

In [22]:
data_path = dm.get_save_directory(datetime)
data_path = os.path.join(data_path, 'objects')

print(os.listdir(data_path))

['book', 'chair', 'cup', 'dining table', 'keyboard', 'knife', 'laptop', 'person', 'potted plant', 'sink', 'sports ball']


In [55]:
ob_name = 'sink'
ob_path = os.path.join(data_path, ob_name)

In [56]:
files = os.listdir(ob_path)
print(len(files))
filename = files[0]
csv_path = os.path.join(ob_path, filename)

716


In [57]:
np_pc = np.loadtxt(csv_path, delimiter=',')

In [58]:
print(np_pc.shape)
np_pc.mean(0)

(19663, 3)


array([-1573.59131633,  -592.57718875,  3042.5855668 ])

In [59]:
pc = np.rollaxis(np_pc, 1)
print(len(pc[0]))
print(len(pc[1]))

19663
19663


In [60]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [61]:
%matplotlib widget

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pc[0], pc[1], pc[2])

plt.show()

FigureCanvasNbAgg()

In [62]:
%matplotlib widget

plt.scatter(pc[0], pc[1])
plt.xlabel("x")
plt.ylabel("y")
plt.show()

FigureCanvasNbAgg()

In [63]:
# Translation matrix
P_matrix_filename = os.path.join('../static_data', 'T.csv')
P = np.loadtxt(P_matrix_filename, delimiter=',')

def convert2world(P, coord):
    '''
    Convert from Camera coordniate to World coordinate (according to P)
    '''
    _coord = np.concatenate([np.asarray(coord), [1.000]])
    _P = np.array(P)
    #FIXME: Remove this when P is fixed
    rotate = np.array([[1,0,0,0], [0,-1,0,0],[0,0,-1,0],[0,0,0,1]])
    n = rotate.dot(_coord)
    return _P.dot(n)[:3]

In [64]:
ordered_files = dm.natural_sort(files)
mean_of_points = np.zeros((len(ordered_files), 3))
for i, fn in enumerate(ordered_files):
    csv_path = os.path.join(ob_path, fn)
    points = np.loadtxt(csv_path, delimiter=',')
    mean = points.mean(0)
    real_world = convert2world(P, mean)
    mean_of_points[i] = real_world

In [65]:
mp = np.asarray(mean_of_points)
mp = np.rollaxis(mp, 1)
mp.shape

(3, 716)

In [71]:
%matplotlib widget

plt.subplot(2, 2, 1)
plt.plot(mp[0], mp[1])
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.suptitle('Movement of object', fontsize=16)

plt.subplot(2, 2, 2)
plt.scatter(mp[0], mp[1])
plt.xlabel("x")
plt.ylabel("y")
plt.grid(True)
plt.show()

FigureCanvasNbAgg()

In [67]:
xedges = [i for i in range(-2000, 2000, 50)]
yedges = [i for i in range(-2000, 2000, 50)]

x = mp[0][~np.isnan(mp[0])]
y = mp[1][~np.isnan(mp[1])]
z = mp[2][~np.isnan(mp[2])]

In [68]:
H, xedges, yedges = np.histogram2d(x, y, bins=(xedges, yedges))
H = H.T

In [69]:
%matplotlib widget

fig = plt.figure()

# imshow:

# ax = fig.add_subplot(131, title='imshow: square bins')
# plt.imshow(H, interpolation='nearest', origin='low',
#            extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])


# pcolor mesh:

ax = fig.add_subplot(111, title='pcolormesh: actual edges',
                     aspect='equal')
X, Y = np.meshgrid(xedges, yedges)
ax.pcolormesh(X, Y, H)

# non uniform image:

# ax = fig.add_subplot(111, title='NonUniformImage: interpolated',
#                      aspect='equal', xlim=xedges[[0, -1]], ylim=yedges[[0, -1]])
# im = mpl.image.NonUniformImage(ax, interpolation='bilinear')
# xcenters = (xedges[:-1] + xedges[1:]) / 2
# ycenters = (yedges[:-1] + yedges[1:]) / 2
# im.set_data(xcenters, ycenters, H)
# ax.images.append(im)
plt.show()

FigureCanvasNbAgg()

In [70]:
plt.savefig('person_hist.png')